In [1]:
import pandas as pd

In [2]:
team_per100 = pd.read_csv('../Data/Team/2018/team_per100.csv')
opp_per100 = pd.read_csv('../Data/Team/2018/opp_per100.csv')
misc = pd.read_csv('../Data/Team/2018/misc.csv')

shooting_col_names = ['Rk', 'Team', 'G', 'MP', 'FG%', 'Dist.',
                      'Pct_FGA_by_dist_2P' , 'Pct_FGA_by_dist_0-3', 'Pct_FGA_by_dist_3-10', 'Pct_FGA_by_dist_10-16', 'Pct_FGA_by_dist_16-3pt', 'Pct_FGA_by_dist_3P',
                     'FGP_by_dist_2P' , 'FGP_by_dist_0-3', 'FGP_by_dist_3-10', 'FGP_by_dist_10-16', 'FGP_by_dist_16-3pt', 'FGP_by_dist_3P',
                     '2FG_%assisted','2FG_dunks_%FGA','2FG_dunks_made','2FG_layups_%FGA','2FG_layups_made',
                     '3FG_%assisted','3FG_corner_%3PA','3FG_corner_3P%','3FG_heaves_att','3FG_heaves_made']

team_shooting = pd.read_csv('../Data/Team/2018/team_shooting.csv', names = shooting_col_names, skiprows = 2)
opp_shooting = pd.read_csv('../Data/Team/2018/opp_shooting.csv', names = shooting_col_names, skiprows = 2)

In [3]:
## Renaming OPP columns to ahve prefix 'opp_'
opp_per100 = opp_per100.add_prefix('opp_')
opp_shooting = opp_shooting.add_prefix('opp_')